In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.4 MB/s eta 0:00:00


"JSON mode" is a mode to force LLM output to JSON format. Available in "gemini-1.5-pro" and "gemini-1.5-flash".

Additionally, gemini-1.5-pro supports "constrained decoding", which allows you to pass a schema object (or equivalent Python type) at inference time and the output will strictly follow that schema.

In [2]:
from google.colab import userdata
import google.generativeai as genai


GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
model = genai.GenerativeModel(
    "gemini-1.5-flash",
    generation_config={"response_mime_type": "application/json"}
)

In [4]:
prompt = """Using the following JSON schema,
Please list 5 popular cookie recipes.
Recipe = {'recipe_name': str}
Return: list[Recipe]"""


raw_response = model.generate_content(prompt)
print(raw_response.text)

[{"recipe_name": "Chocolate Chip Cookies"}, {"recipe_name": "Oatmeal Raisin Cookies"}, {"recipe_name": "Sugar Cookies"}, {"recipe_name": "Peanut Butter Cookies"}, {"recipe_name": "Snickerdoodles"}]



In [5]:
import json
response = json.loads(raw_response.text)
print(response)

[{'recipe_name': 'Chocolate Chip Cookies'}, {'recipe_name': 'Oatmeal Raisin Cookies'}, {'recipe_name': 'Sugar Cookies'}, {'recipe_name': 'Peanut Butter Cookies'}, {'recipe_name': 'Snickerdoodles'}]


## JSON output with constrained decoding

In [6]:
import typing_extensions as typing

# Schema object definition
class Recipe(typing.TypedDict):
    recipe_name: str

Specify the JSON format in the schema object and execute inference.

Specify "application/json" for response_mime_type of
generation_config() and the schema object for response_schema .

In [7]:
# Direct JSON output with constrained decoding
result = model.generate_content(
    "Please list 5 popular cookie recipes.",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema = list[Recipe]
    ),
    request_options={"timeout": 600},  # timeout
)
print(result.text)

{"recipes": [{"name": "Chocolate Chip Cookies", "ingredients": ["1 cup (2 sticks) unsalted butter, softened", "3/4 cup granulated sugar", "3/4 cup packed light brown sugar", "2 large eggs", "2 teaspoons pure vanilla extract", "2 1/4 cups all-purpose flour", "1 teaspoon baking soda", "1 teaspoon salt", "2 cups chocolate chips"], "instructions": ["Preheat oven to 375 degrees F (190 degrees C).", "In a large bowl, cream together the butter, granulated sugar, and brown sugar until light and fluffy.", "Beat in the eggs one at a time, then stir in the vanilla.", "In a separate bowl, whisk together the flour, baking soda, and salt.", "Gradually add the dry ingredients to the wet ingredients, mixing until just combined.", "Stir in the chocolate chips.", "Drop by rounded tablespoons onto ungreased baking sheets.", "Bake for 9-11 minutes, or until edges are golden brown.", "Let cool on baking sheets for a few minutes before transferring to a wire rack to cool completely."] }, {"name": "Oatmeal R